# Encrypt Client

In [93]:
import requests

## 1. Get public key from server

In [94]:
public_key = requests.get('http://localhost:8080/getPublicKey')
print(public_key)
print()
print(public_key.text)

<Response [200]>

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvj2YjA2ElbtszsUQCjzB\nMz4Yfc2em9D7Kn8Y2lbcUT0KYFEOlylpq8b9785D1nCS08KkzVOJzZlG5ufCSjVA\nbspfTv3+D6sf//gezEeHAH5fnBb7zY5YaERzNOYBcH+RwUFhv5MK/fWXVV/v487u\neQTxyn0uZquOPfEVas9KYgD3yr6/j6idsPnQn7/z7V4QbwTTy9Ox234MX3FTblHX\n276c23J/zGwNN08ZwhffUkxuZoChXC0tqQLwkr2RmRnqE95YE+vyOqs6fot2h1Wy\nCiexirGs4SyfxVHCiAzxsasFoMtsBP2ANkMiTiWb8Us3+D96Zx84NWVa0h3cIsmf\nuQIDAQAB\n-----END PUBLIC KEY-----\n'


## 2. Generate a symmetric key and use the server public key to encrypt it

In [95]:
import os
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_public_key

def encrypt_symmetric_key_with_public_key(public_key_pem, symmetric_key):
    public_key = load_pem_public_key(public_key_pem)

    encrypted_key = public_key.encrypt(
        symmetric_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_key

# 客户端生成一个随机的AES密钥
aes_key = os.urandom(32)  # 例如，使用32字节的密钥

print(('\n' + public_key.text[2:-1].replace("\\n", "\n")).encode())

# 加密AES密钥
encrypted_aes_key = encrypt_symmetric_key_with_public_key(('\n' + public_key.text[2:-1].replace("\\n", "\n")).encode(), aes_key)

print()
print(aes_key)
print()
print(encrypted_aes_key)

b'\n-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAvj2YjA2ElbtszsUQCjzB\nMz4Yfc2em9D7Kn8Y2lbcUT0KYFEOlylpq8b9785D1nCS08KkzVOJzZlG5ufCSjVA\nbspfTv3+D6sf//gezEeHAH5fnBb7zY5YaERzNOYBcH+RwUFhv5MK/fWXVV/v487u\neQTxyn0uZquOPfEVas9KYgD3yr6/j6idsPnQn7/z7V4QbwTTy9Ox234MX3FTblHX\n276c23J/zGwNN08ZwhffUkxuZoChXC0tqQLwkr2RmRnqE95YE+vyOqs6fot2h1Wy\nCiexirGs4SyfxVHCiAzxsasFoMtsBP2ANkMiTiWb8Us3+D96Zx84NWVa0h3cIsmf\nuQIDAQAB\n-----END PUBLIC KEY-----\n'

b'6\xb5A1\x05\xa5\xaf4>\x9bg\xc1v\x83H$/\x8c\x11\xcf\xaa\x8c\xac\x85[\xddO\x11\x9b=\x04['

b'\r\xc6\xf3X\xe7\x96\xfc\xc1:n8k y\x15\x92\xe3\xac~\xf1\x94Q\xdf7\xdc\x89\x03\xa3w\x88\xe8\xdc\xc8n\xa0W\x99\xed\xe9\xdcc\xa8\x08^\xd9\xa74\x19\xacW\x04\x8et\xd5y\xb9\x98M\xe7\x8d#\x88\x90\xe5\xc8\xfb\xf7~\x81\xa5$\x18D_\x8e\x83j{\'O\xadw\xe6Ui{\x89\xf6\x97\xdc\x01\x84TJ\x99\xac\x82x)=qO~\xac\xf1\xd1\xd5\xeb\xf3\x95(\x02\x03\x9f\x9f\x81\x9d\x93\xae\x0f\xaeX\x18\x00w\xfd8\x9f\xbe\xd2]\xdc"\x01\x14\xe7+\x85\x18]\xa0\x9d\xbb\xbe\x14\xd6\xf4

## 3. Send encrypted key to server

In [96]:
res = requests.post('http://localhost:8080/sendEncryptKey', data=encrypted_aes_key)
print(res)
print(res.text)

<Response [200]>
Send Success!


## 4. Encrypted communication with the server

In [97]:
import AES
import io

with open("tmp/a.txt", 'rb') as file:
    file_data = file.read()
encrypted_data = AES.aes_encrypt(aes_key, file_data)
# 创建一个内存中的字节流
encrypted_file = io.BytesIO(encrypted_data)
files = {'file': ('a.txt', encrypted_file)} # 使用内存中的字节流

headers = {"Authorization": "Basic Y2xpZW50MToxMjM=", "X-Content-Encrypted": "true"}
res = requests.post(url='http://127.0.0.1:8080/upload?path=client1/', headers=headers, files=files)
print(res)
print(AES.aes_decrypt(aes_key, res.content))

headers = {"Authorization": "Basic Y2xpZW50MToxMjM=", "X-Content-Encrypted": "true"}
res = requests.get(url='http://127.0.0.1:8080/client1/a.txt', headers=headers)
print(AES.aes_decrypt(aes_key, res.content))

<Response [200]>
b'Upload success!'
b'sadfsdfasdf'
